### Summary as of 27 Jan 
- I tried different package to read pdf, and pdftotext works best for the transcripts files
- There are two format types. One is fancier. 
- My next step is to parse the fancier files.

### Summary as of 1st Feb
**Parse fancy files**

- the basic unit: each paragraph spoken by one person (i.e. the content divided by dotted line)
- clean those words that are repeated and obviously useless
- Words appeared in the front page and CORPORATE PARTICIPANTS session are repeated and contain no information

**Next step**

- topic modeling
- optimize text preprocessing (if needed)

In [270]:
import pdftotext
import os
import pandas as pd

def is_fancy(filepath):
    check_words = ["Corrected Transcript",
                  "www.callstreet.com",
                  "FactSet CallStreet, LLC"]
    pdfFileObj = open(filepath,'rb')
    pdf = pdftotext.PDF(pdfFileObj)
    text = "\n\n".join(pdf)
    if all([ check_word in text for check_word in check_words]):
        return("fancy")
    else:
        return("plain")
    
def get_fp_cp(fp):
    output = list()
    for item in fp[fp.index("CORPORATE PARTICIPANTS"):]:
        if "Walmart" in item:
            for x in item.split("."):
                output = output + x.split(",")
            for x in item.split("-"):
                output = output + x.split("   ")
            for x in item.split("-"):
                output = output + x.split(".")
            for x in item.split("   "):
                output = output + x.split(",")
        else: 
            output = output + item.split()
    return([item.strip() for item in output if len(item) > 1]) 

In [2]:
path = os.getcwd() + '\data\Transcripts2019'
files = os.listdir(path)

In [9]:
files = pd.DataFrame(files, columns=["filename"])
files["type"] = files["filename"].map(lambda x: is_fancy(path + "\\" + x))
files

,filename,type
0,20190219_earningcall_2019Q4.pdf,fancy
1,20190305_conference.pdf,plain
2,20190306_conference.PDF,fancy
3,20190312_conference.pdf,fancy
4,20190516_earningcall_2020Q1.pdf,plain
5,20190530_conference.pdf,fancy
6,20190607_investmentcommunity.pdf,plain
7,20190611_conference.pdf,fancy
8,20190815_earningcall_2020Q2.pdf,plain
9,20190904_conference.pdf,fancy


### Parse fancy files
- the basic unit: each paragraph spoken by one person (i.e. the content divided by dotted line)
- clean those words that are repeated and obviously useless
- Words appeared in the front page and CORPORATE PARTICIPANTS session are repeated and contain no information

In [287]:
def parse_fancy_para(filepath):
    """ return a dataframe, each row is a paragraph spoken by one person
    """ 
    pdfFileObj = open(filepath,'rb')
    pdf = pdftotext.PDF(pdfFileObj)
    text = "\n\n".join(pdf)

    tmp = text.split("........................................\
..............................................................\
..............................................................\
..................................................................................")

    # process words appeared in the front page and CORPORATE PARTICIPANTS section
    fp = [item.strip() for item in tmp[0].split("\r\n") if len(item)>=1]

    to_be_removed = [fp[0], fp[1], fp[3]] + \
                    fp[2].split() + \
                    [x for x in fp[5].split(" ") if len(x) >= 1] + \
                    get_fp_cp(fp)

    to_be_removed_df = pd.DataFrame(set(to_be_removed))
    to_be_removed_df["len"] = to_be_removed_df[0].map(lambda x: len(x))
    to_be_removed_df = to_be_removed_df.sort_values(["len"], ascending = False)

    for item in to_be_removed_df[0]:
        for i in range(1, len(tmp)):
            tmp[i] = tmp[i].replace(item, '')
            tmp[i] = tmp[i].replace("\r\n", '').lstrip(", ")

    return(pd.DataFrame(tmp))

In [288]:
fancy = files[ files["type"] == "fancy" ]

test = pd.DataFrame()
for filename in fancy["filename"]:
    filepath = path + "\\" + filename
    test = pd.concat([test, parse_fancy_para(filepath)], ignore_index=True)

In [289]:
test

,0
0,...
1,OTHER Simeon Ari Gutman ...
2,Thank you. Good morning and welcome to 's four...
3,"Good morning, everyone, and thanks for joining..."
4,"You bet. Thanks, Doug. Good morning, everybody..."
5,QUESTION AND ANSWER SECTIONOperator: Thank you...
6,"Simeon Ari GutmanAnalyst, Morgan Stanley & Co...."
7,"AYeah, thanks, Simeon. We've talked a lot over..."
8,10 ...
9,"AYeah, Simeon, this is Doug. I think the headl..."


### Below is working space: trying to optimize deletion of CORPORATE PARTICIPANTS section

In [285]:
fancy = files[ files["type"] == "fancy" ]
filename = fancy["filename"].iloc[1]
filepath = path + "\\" + filename

pdfFileObj = open(filepath,'rb')
pdf = pdftotext.PDF(pdfFileObj)
text = "\n\n".join(pdf)

tmp = text.split("........................................\
..............................................................\
..............................................................\
..................................................................................")

# process words appeared in the front page and CORPORATE PARTICIPANTS section
fp = [item.strip() for item in tmp[0].split("\r\n") if len(item)>=1]

to_be_removed = [fp[0], fp[1], fp[3]] + \
                fp[2].split() + \
                [x for x in fp[5].split(" ") if len(x) >= 1] + \
                get_fp_cp(fp)

to_be_removed_df = pd.DataFrame(set(to_be_removed))
to_be_removed_df["len"] = to_be_removed_df[0].map(lambda x: len(x))
to_be_removed_df = to_be_removed_df.sort_values(["len"], ascending = False)

for item in to_be_removed_df[0]:
    for i in range(1, len(tmp)):
        tmp[i] = tmp[i].replace(item, '')
        tmp[i] = tmp[i].replace("\r\n", '').lstrip(", ")

In [284]:
to_be_removed_df = pd.DataFrame(set(to_be_removed))
to_be_removed_df["len"] = to_be_removed_df[0].map(lambda x: len(x))
to_be_removed_df = to_be_removed_df.sort_values(["len"], ascending = False)
to_be_removed_df[0]

5     UBS Securities LLC                            ...
10    Analyst, UBS Securities LLC                   ...
13       President & Chief Executive Officer-Walmart US
25             Chief Financial Officer-Walmart US & EVP
20              UBS Global Consumer & Retail Conference
36                  President & Chief Executive Officer
0                       Walmart US & EVP, Walmart, Inc.
1                        Walmart US & EVP, Walmart, Inc
23                          Analyst, UBS Securities LLC
7                             Walmart US, Walmart, Inc.
6                              Walmart US, Walmart, Inc
14                              Chief Financial Officer
33                                 Corrected Transcript
35                                   www.callstreet.com
32                                   UBS Securities LLC
4                                         1-877-FACTSET
2                                          PARTICIPANTS
11                                          06-M